#### Q1. Install the package
To get started with MLflow you'll need to install the appropriate Python package.

For this we recommend creating a separate Python environment, for example, you can use conda environments, and then install the package there with pip or conda.

Once you installed the package, run the command `mlflow --version` and check the output.

What's the version that you have?

Ans: 2.11.3

In [2]:
!mlflow --version

mlflow, version 2.11.3


#### Q2. Download and preprocess the data

We'll use the Green Taxi Trip Records dataset to predict the amount of tips for each trip.

Download the data for January, February and March 2022 in parquet format from here.

Use the script `preprocess_data.py` located in the folder homework to preprocess the data.

The script will:
* load the data from the folder `<TAXI_DATA_FOLDER>` (the folder where you have downloaded the data),
* fit a `DictVectorizer` on the training set (January 2022 data),
* save the preprocessed datasets and the `DictVectorizer` to disk.

Your task is to download the datasets and then execute this command:

```
python preprocess_data.py --raw_data_path <TAXI_DATA_FOLDER> --dest_path ./output
```

Tip: go to `02-experiment-tracking/homework/` folder before executing the command and change the value of `<TAXI_DATA_FOLDER>` to the location where you saved the data.

So what's the size of the saved `DictVectorizer` file?

Ans: 154 kB (actual is 151 KB)

In [5]:
data_path = r'C:\Users\Jiet_Mao_Lim\IdeaProjects\epam_learn\mlops-zoomcamp\01-intro\data'
script_path = r'C:\Users\Jiet_Mao_Lim\IdeaProjects\epam_learn\mlops-zoomcamp\cohorts\2023\02-experiment-tracking\homework\preprocess_data.py'

In [6]:
!python {script_path} --raw_data_path {data_path} --dest_path ./output

#### Q3. Train a model with autolog

We will train a `RandomForestRegressor` (from Scikit-Learn) on the taxi dataset.

We have prepared the training script `train.py` for this exercise, which can be also found in the folder `homework`. 

The script will:

* load the datasets produced by the previous step,
* train the model on the training set,
* calculate the RMSE score on the validation set.

Your task is to modify the script to enable **autologging** with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked. 

Tip 1: don't forget to wrap the training code with a `with mlflow.start_run():` statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the `max_depth` parameter:

Ans: 10

In [7]:
train_script_path = r'C:\Users\Jiet_Mao_Lim\IdeaProjects\epam_learn\mlops-zoomcamp\cohorts\2023\02-experiment-tracking\homework\train.py'

In [9]:
!python {train_script_path} --data_path ./output

2024/04/06 20:53:08 INFO mlflow.tracking.fluent: Experiment with name '02-experiment-tracking-homework' does not exist. Creating a new experiment.
2024/04/06 20:53:08 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2024/04/06 20:53:08 WARNING mlflow.spark: With Pyspark >= 3.2, PYSPARK_PIN_THREAD environment variable must be set to false for Spark datasource autologging to work.
2024/04/06 20:53:08 INFO mlflow.tracking.fluent: Autologging successfully enabled for pyspark.
2024/04/06 20:53:09 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/04/06 20:53:23 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Jiet_Mao_Lim\AppData\Local\anaconda3\envs\mlops-zoomcamp\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils."
C:\Users\Jiet_Mao_Lim\AppData\Local\anaconda3\envs\mlops-zoomcam

#### Q4. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the `RandomForestRegressor` using `optuna`. 
We have prepared the script `hpo.py` for this exercise. 

Your task is to modify the script `hpo.py` and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the `objective` function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called `random-forest-hyperopt` to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

* the list of hyperparameters that are passed to the `objective` function during the optimization,
* the RMSE obtained on the validation set (February 2022 data).

What's the best validation RMSE that you got?

Ans: 2.45

In [ ]:
artifact_path = 'file:///C:/Users/Jiet_Mao_Lim/IdeaProjects/epam_learn/mlops-zoomcamp/02-experiment-tracking/artifacts'

In [10]:
!mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root {artifact_path}

^C


In [11]:
hpo_script_path = r'C:\Users\Jiet_Mao_Lim\IdeaProjects\epam_learn\mlops-zoomcamp\cohorts\2023\02-experiment-tracking\homework\hpo.py'

In [13]:
!python {hpo_script_path}

2024/04/06 21:12:26 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
[I 2024-04-06 21:12:26,067] A new study created in memory with name: no-name-02fdc4d4-8993-43e6-a4cc-d76b154f6118
C:\Users\Jiet_Mao_Lim\IdeaProjects\DataTalksClub\mlops-zoomcamp\cohorts\2023\02-experiment-tracking\homework\hpo.py:39: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'n_estimators': trial.suggest_int('n_estimators', 10, 50, 1),
C:\Users\Jiet_Mao_Lim\IdeaProjects\DataTalksClub\mlops-zoomcamp\cohorts\2023\02-experiment-tracking\homework\hpo.py:40: FutureWarning: suggest_int() got {'step'} as positional arguments but they were expected to be given as keyword arguments.
  'max_depth': trial.suggest_int('max_depth', 1, 20, 1),
C:\Users\Jiet_Mao_Lim\IdeaProjects\DataTalksClub\mlops-zoomcamp\cohorts\2023\02-experiment-tracking\homework\hpo.py:41: FutureWarning: sugges

#### Q5. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. 
In this exercise, you'll promote the best model to the model registry. We have prepared a script called `register_model.py`, which will check the results from the previous step and select the top 5 runs. 
After that, it will calculate the RMSE of those models on the test set (March 2022 data) and save the results to a new experiment called `random-forest-best-models`.

Your task is to update the script `register_model.py` so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tips for MLflow:

* you can use the method `search_runs` from the `MlflowClient` to get the model with the lowest RMSE,
* to register the model you can use the method `mlflow.register_model` and you will need to pass the right `model_uri` in the form of a string that looks like this: `"runs:/<RUN_ID>/model"`, and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

Ans: 2.185

In [14]:
model_script_path = r'C:\Users\Jiet_Mao_Lim\IdeaProjects\epam_learn\mlops-zoomcamp\cohorts\2023\02-experiment-tracking\homework\register_model.py'

In [17]:
!python {model_script_path}

2024/04/06 21:32:51 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/04/06 21:32:55 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "C:\Users\Jiet_Mao_Lim\AppData\Local\anaconda3\envs\mlops-zoomcamp\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils."
C:\Users\Jiet_Mao_Lim\AppData\Local\anaconda3\envs\mlops-zoomcamp\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
C:\Users\Jiet_Mao_Lim\AppData\Local\anaconda3\envs\mlops-zoomcamp\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'ro

#### Q6. Model metadata

Now explore your best model in the model registry using UI. What information does the model registry contain about each model?

* Version number
* Source experiment
* Model signature
* All the above answers are correct

Ans: All the above answers are correct